In [13]:
import numpy as np
import pandas as pd
import re
import json 
import pathlib as pl

# Data Ingestion
Data are downloaded from the course drive folder, link is not reported to avoid external access . The first step is to read the two csv files as pandas Dataframes

In [2]:
credits = pd.read_csv("data/tmdb_5000_credits.csv")
movies = pd.read_csv("data/tmdb_5000_movies.csv")

In [3]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
credits.shape

(4803, 4)

In [5]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [6]:
movies.shape

(4803, 20)

# Data Preprocessing
Let's proceed with some preprocessing, like checking for missing values, better exploring the schema of the data and looking for identifiers.

In [7]:
# MISSING VALUES
credits_missing = credits.shape[0] - credits.dropna(how='any').shape[0]
movies_missing = movies.shape[0] - movies.dropna(how='any').shape[0]
print("The credits table has",credits_missing,"rows with at least one missing value.")
print("The movies table has",movies_missing,"rows with at least one missing value.")

The credits table has 0 rows with at least one missing value.
The movies table has 3310 rows with at least one missing value.


In [8]:
# CHECK THAT MOVIE_ID CAN BE USED AS PRIMARY KEY
credits_movieid = len(credits['movie_id'].unique())
movies_movieid = len(movies['id'].unique())
print("There are",credits_movieid,"unique movie id values in the credits table")
print("There are",movies_movieid,"unique movie id values in the movies table")

There are 4803 unique movie id values in the credits table
There are 4803 unique movie id values in the movies table


In [9]:
# SOME TITLES CORRESPOND TO MORE THAN ONE MOVIE_ID
a = credits.groupby('title')['movie_id'].size()
a.index[a > 1]

Index(['Batman', 'Out of the Blue', 'The Host'], dtype='object', name='title')

In [10]:
credits.loc[credits['title']== 'Batman']

,movie_id,title,cast,crew
1359,268,Batman,"[{""cast_id"": 5, ""character"": ""Jack Napier/The ...","[{""credit_id"": ""52fe422fc3a36847f800aa4b"", ""de..."
4267,2661,Batman,"[{""cast_id"": 17, ""character"": ""Batman / Bruce ...","[{""credit_id"": ""52fe4363c3a36847f80509a7"", ""de..."


In [11]:
credits.loc[credits['title']=='Out of the Blue']

,movie_id,title,cast,crew
3647,39269,Out of the Blue,"[{""cast_id"": 2, ""character"": ""Don"", ""credit_id...","[{""credit_id"": ""52fe47099251416c9106826f"", ""de..."
3693,10844,Out of the Blue,"[{""cast_id"": 12, ""character"": ""Nick Harvey"", ""...","[{""credit_id"": ""52fe43c19251416c7501cceb"", ""de..."


In [12]:
credits.loc[credits['title']=='The Host']

,movie_id,title,cast,crew
972,72710,The Host,"[{""cast_id"": 52, ""character"": ""Melanie Stryder...","[{""credit_id"": ""52fe487bc3a368484e0fa919"", ""de..."
2877,1255,The Host,"[{""cast_id"": 3, ""character"": ""Park Gang-du"", ""...","[{""credit_id"": ""52fe42eac3a36847f802ca6b"", ""de..."


# First request
## For each movie, computer the number of cast members.
We first select from the credits table only the necessary attributes.

In [15]:
first = credits[['title','cast']]

Cells in the *cast* column contain strings which represent a list of many dictionaries. Each dictionary contains information specific to a cast member, so that each list can be interpreted as a json document describing the cast of the movie. Finally, since we need to compute the number of cast members, we compute the length of the list.

In [15]:
#THE assign METHOD CREATES A COPY OF THE DATAFRAME IN ORDER TO AVOID SettingWithCopyWarning
first = first.assign(n_cast_members = first['cast'].apply(json.loads).apply(lambda x: len(x)) )

# Second request
## How many movies do not have a homepage?
We select the homepage attribute of the **movies** table. We keep track of the total number of movies, which we already know is $4803$ with the variable `homepages_total`

In [16]:
second = movies['homepage']
homepages_total = second.shape[0]

Now we can drop all the null values in the homepage column, we then store the number of movies with no homepage reported in the variable `null_homepages`.

In [17]:
#CHECK FOR NULL VALUES
second = second.dropna(axis=0)
null_homepages = homepages_total - second.shape[0]

It could be that instead of a `NaN` value, movies without a homepage are indicated through some meaningful string, such as **none**,**missing** or **unavailable**. To cover these cases a regex is used and if one of the keywords is found, its index is added to `no_homepage_list`. 

In [18]:
equivalent_null_list = ['none','missing','unavailable']
no_homepage_list =[]
for website in second:
    if re.compile('|'.join(equivalent_null_list),re.IGNORECASE).search(website): #re.IGNORECASE is used to ignore case
        no_homepage_list.append(second[second == website].index[0])
print(no_homepage_list)

[]


Since our regex has provided no matches, we believe that our initial computation is enough to solve the problem.

In [19]:
print("Number of movies without an homepage: ",null_homepages)

Number of movies without an homepage:  3091


# Third request
## For each year, how many movies do not have a homepage?
We begin by removing those movies that are missing a release date

In [42]:
#REMOVE MOVIES WITH NO RELEASE DATE, copy TO AVOID SettingWithCopyWarning
third = movies.dropna(subset=['release_date']).copy()
third.shape

(4802, 20)

The *release_date* attribute can be read as `DatetimeIndex` to extract only the year of release. 

In [43]:
third['release_year'] = pd.DatetimeIndex(third['release_date']).year
#VISUALIZE INTEGER YEAR VALUES
third['release_year'] = third['release_year'].apply(lambda x: int(x))

We now only the *release_year* and *homepage* attributes. The Dataframe is then grouped by the year and through the `isnull()` method a list of `True` or `False` values is computed, if the homepage is present or not, respectively. Finally we perform a `sum()` over the elements of the list, in fact `True = 1` and `False = 0`, to get the total number of movies without an homepage for each year.

In [44]:
#COMPUTE NUMBER OF MOVIES WITH NO HOMEPAGE FOR EACH YEAR
no_homepage_movies = third[['release_year','homepage']].groupby('release_year')['homepage'].apply(lambda x: x.isnull().sum())
no_homepage_movies.head()

release_year
1916    1
1925    1
1927    1
1929    2
1930    1
Name: homepage, dtype: int64

# Fourth request
## Extract the domain of each homepage.
The *homepage* attribute of the **movies** table is selected and all null values are dropped. The indexes are then reset to provide compatibility for future merging with a domain series.

In [108]:
fourth = movies['homepage'].dropna()
homepage_series = fourth.reset_index(drop = True)

We make use of regex to extract the domain of the homepage. A first step is to verify that each value of the homepage attribute contains only one URL.

In [109]:
multiple_homepages = homepage_series.apply(lambda x: re.split('; |, | ',x))
index = np.arange(len(multiple_homepages))
for i in index:
    if len(multiple_homepages[i]) > 1:
        print(multiple_homepages[i], i)

['http://www.cargoderfilm.ch', 'http://cargothemovie.com'] 1403


So there actually is one entry with two homepages. We then check that every string starts with *http* so that we are sure they are valid urls. If this is the case, we can explore each homepage to find its domain. 

In [110]:
http = 0
domain_list = []
i=0
#CHECK IF EVERY URL STARTS WITH AT LEAST http, SO THAT WE ARE SURE WE CAN SEARCH FOR DOMAINS INSIDE SLASHES, e.g. http://www.domainexample.example/otherstuff
for x in homepage_series:
    if re.search('^http',x):
        http = http + 1
if http == homepage_series.shape[0]:
    for x in fourth:
        #string begins with http(s could be there or not) then it goes ://
        #then we have the domain, it can be a string of any length > 1, but can't include a ] character
        #the URL can go on with a / or could end there.
        m = re.search('(^https?:\/\/)(?P<domain>[^\[]*?)(\/|$)',x)
        domain_list.append(m.group('domain'))
else:
    print("At least one URL begins without http / https. Dig deeper.")

The list of domains created is now transformed into a `pandas.Series` to be merged with the initial homepage series.

In [111]:
#TRANSFORM THE DOMAIN LIST INTO A SERIES TO BE MERGED WITH THE FULL HOMEPAGE URLs
domain_series = pd.Series(domain_list,name='domain')


In [112]:
#MERGE HOMEPAGES WITH RESPECTIVE DOMAIN
domain_df = pd.concat([homepage_series,domain_series],axis=1)
domain_df.head()

,homepage,domain
0,http://www.avatarmovie.com/,www.avatarmovie.com
1,http://disney.go.com/disneypictures/pirates/,disney.go.com
2,http://www.sonypictures.com/movies/spectre/,www.sonypictures.com
3,http://www.thedarkknightrises.com/,www.thedarkknightrises.com
4,http://movies.disney.com/john-carter,movies.disney.com


# Fifth request
## Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).
We first have to find a common feature for all non-normalized attributes. We discover that attributes such as *cast*, *crew* or *spoken_languages*, all formatted as json documents, are of type `str`.

In [19]:
fifth_credits = credits.copy()
for column in fifth_credits.columns:
    print(column, type(fifth_credits[column][0]))

movie_id <class 'numpy.int64'>
title <class 'str'>
cast <class 'str'>
crew <class 'str'>


In [20]:
fifth_movies = movies.copy()
for column in fifth_movies.columns:
    print(column, type(fifth_movies[column][0]))

budget <class 'numpy.int64'>
genres <class 'str'>
homepage <class 'str'>
id <class 'numpy.int64'>
keywords <class 'str'>
original_language <class 'str'>
original_title <class 'str'>
overview <class 'str'>
popularity <class 'numpy.float64'>
production_companies <class 'str'>
production_countries <class 'str'>
release_date <class 'str'>
revenue <class 'numpy.int64'>
runtime <class 'numpy.float64'>
spoken_languages <class 'str'>
status <class 'str'>
tagline <class 'str'>
title <class 'str'>
vote_average <class 'numpy.float64'>
vote_count <class 'numpy.int64'>


In [21]:
for column in fifth_credits.columns:
    col_value = fifth_credits[column][0]
    if isinstance(col_value, str):
            try:
                column_values = fifth_credits[column].apply(json.loads)
                column_final = pd.DataFrame()
                column_index = np.arange(4803)
                for index in column_index:
                    value_df = pd.DataFrame(column_values[index])
                    value_df['movie_id'] = credits['movie_id'][index]
                    column_final = pd.concat([column_final,value_df])
                column_final = column_final.reset_index(drop=True)
                pl.Path('FIFTH_POINT/CREDITS').mkdir(parents=False, exist_ok=True)
                path = "FIFTH_POINT/CREDITS/{0}.csv".format(column)
                column_final.to_csv(path_or_buf = path)
                print("Succesfully written normalized table to ", path)
            except:
                print("Column",column,"is not formatted as a json document")

Column title is not formatted as a json document
Succesfully written normalized table to  FIFTH_POINT/CREDITS/cast.csv
Succesfully written normalized table to  FIFTH_POINT/CREDITS/crew.csv


In [22]:
for column in fifth_movies.columns:
    col_value = fifth_movies[column][0]
    if isinstance(col_value, str):
            try:
                column_values = fifth_movies[column].apply(json.loads) 
                column_final = pd.DataFrame()
                column_index = np.arange(4803)
                for index in column_index:
                    value_df = pd.DataFrame(column_values[index])
                    value_df['movie_id'] = movies['id'][index]
                    column_final = pd.concat([column_final,value_df])
                column_final = column_final.reset_index(drop=True)
                pl.Path('FIFTH_POINT/MOVIES').mkdir(parents=False, exist_ok=True)
                path = "FIFTH_POINT/MOVIES/{0}.csv".format(column)
                column_final.to_csv(path_or_buf = path)
                print("Succesfully written normalized table to ", path)
            except:
                print("Column",column,"is not formatted as a json document")

Succesfully written normalized table to  FIFTH_POINT/MOVIES/genres.csv
Column homepage is not formatted as a json document
Succesfully written normalized table to  FIFTH_POINT/MOVIES/keywords.csv
Column original_language is not formatted as a json document
Column original_title is not formatted as a json document
Column overview is not formatted as a json document
Succesfully written normalized table to  FIFTH_POINT/MOVIES/production_companies.csv
Succesfully written normalized table to  FIFTH_POINT/MOVIES/production_countries.csv
Column release_date is not formatted as a json document
Succesfully written normalized table to  FIFTH_POINT/MOVIES/spoken_languages.csv
Column status is not formatted as a json document
Column tagline is not formatted as a json document
Column title is not formatted as a json document


The attribute `release_date` is considered to be normalized, since each date corresponds to a single event. If anyway we were interested in retrieving only the year, month or day of release we could easily do so through the following code.

In [90]:
date = pd.DataFrame(movies['release_date'].copy())
date['year'] = pd.DatetimeIndex(date['release_date']).year
date['month'] = pd.DatetimeIndex(date['release_date']).month
date['day'] = pd.DatetimeIndex(date['release_date']).day

# Sixth request
## For each movie, compute the gross margin (difference between revenue and budget)
In this case it is sufficient to select the *revenue* and *budget* attributes and to create a third attribute named *margin* to store the differences between values of the first two attributes. 

In [37]:
movies_margin = movies[['id','title','revenue','budget']].copy()
movies_margin['margin'] = movies_margin['revenue']-movies_margin['budget']
movies_margin.head()

,id,title,revenue,budget,margin
0,19995,Avatar,2787965087,237000000,2550965087
1,285,Pirates of the Caribbean: At World's End,961000000,300000000,661000000
2,206647,Spectre,880674609,245000000,635674609
3,49026,The Dark Knight Rises,1084939099,250000000,834939099
4,49529,John Carter,284139100,260000000,24139100


# Seventh request
## For each movie, compute the number of crew members
Similarly to what we have done for the first request, we can select *movie_id*, *title* and *crew* attributes. On the last one we apply `json.loads` to read information about crew members. The length of the list contained in each cell of the attribute *crew* corresponds to the number of crew members.

In [36]:
seventh = credits[['movie_id','title','crew']].copy()
seventh['n_crew_members'] = seventh['crew'].apply(json.loads).apply(lambda x : len(x))
seventh.head()

,movie_id,title,crew,n_crew_members
0,19995,Avatar,"[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",153
1,285,Pirates of the Caribbean: At World's End,"[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...",32
2,206647,Spectre,"[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...",155
3,49026,The Dark Knight Rises,"[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...",217
4,49529,John Carter,"[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...",132


# Eighth request
## For each movie, compute the number of directors
One of the normalized tables generated for the fifth request contains all the information we need to perform this task. We start by reading the csv file.

In [13]:
crew = pd.read_csv("FIFTH_POINT/CREDITS/crew.csv")

We notice that there is a different number of unique movie ids between the **crew** table and the original **credits** table. We then proceed to investigate on this behaviour.

In [14]:
len(crew['movie_id'].unique()) 

4775

This is because some movies have no crew members reported!

In [15]:
credits[credits.duplicated(subset=['crew'],keep=False)].head()

,movie_id,title,cast,crew
3661,19615,Flying By,"[{""cast_id"": 1, ""character"": ""George"", ""credit...",[]
3670,447027,Running Forever,[],[]
3977,55831,Boynton Beach Club,"[{""cast_id"": 1, ""character"": ""Marilyn"", ""credi...",[]
4068,371085,Sharkskin,[],[]
4105,48382,"The Book of Mormon Movie, Volume 1: The Journey","[{""cast_id"": 1, ""character"": ""Sam"", ""credit_id...",[]


We can compare the number of these movies to the difference between the number of unique movie ids of the two tables to find out that they match. So we cannot find the director of these movies after all.

In [48]:
print(credits[credits.duplicated(subset=['crew'],keep=False)].shape[0])
print(len(credits['movie_id'].unique())  - len(crew['movie_id'].unique()) )

28
28


The next step is to group our data by *movie_id* and *job*. Then we can create a new dataframe with the id and the number of directors of the movie. 

In [49]:
crew_gb  = crew.groupby(['movie_id','job'], as_index = False)
movie_id = []
n_directors = []
for (k1,k2), group in crew_gb: 
    if k2 == 'Director':
        movie_id.append(k1)
        n_directors.append(group.shape[0])
eighth = pd.DataFrame({'movie_id':movie_id, 'n_directors':n_directors})

# Ninth request
## For each language, compute the number of movies where such language is spoken.
We again make use of a normalized table previously created. The data is read then grouped by the language *name*. The `size()` method counts how many entries there are for each group

In [53]:
languages = pd.read_csv("FIFTH_POINT/MOVIES/spoken_languages.csv")
ninth = languages.groupby('name').size()
ninth.head()

name
??????              1
Afrikaans           7
Bahasa indonesia    2
Bamanankan          1
Bosanski            2
dtype: int64

# Tenth request
## For each company and each decade, compute the overall revenue
The useful attributes for this task are *release_date*, *production_companies* and *revenue*. First we drop any movies without a release date. 

In [14]:
tenth = movies[['release_date','production_companies','revenue']].dropna(subset=['release_date']).copy().reset_index(drop=True)
tenth.shape

(4802, 3)

To extract the decade which each movie belongs to we can take the year as done before and divide it by 10. A new column is thus created and the *release_date* attribute is dropped.

In [15]:
tenth['decade'] = pd.DatetimeIndex(tenth['release_date']).year // 10
tenth.drop(['release_date'], axis = 'columns', inplace = True )
tenth.shape

(4802, 3)

Since *production_companies* is formatted as a json document we can exploit code similar to what we have previously used. This time we will also append the *revenue* and the *decade* attributes to the normalized table. Since the *revenue* attribute refers to the movie total revenue, those movies that have more than one production company will have their revenue evenly split among the companies (even though this could not be the real case).

In [18]:
companies = tenth['production_companies'].apply(json.loads)
companies_df = pd.DataFrame()
column_index = np.arange(4802)
for index in column_index: 
    if len(companies[index]) != 0 :
        company_df = pd.DataFrame(companies[index])
        company_df['revenue'] = int(tenth['revenue'][index] / len(companies[index]))
        company_df['decade'] = tenth['decade'][index]    
        companies_df = pd.concat([companies_df,company_df])
companies_df = companies_df.reset_index(drop=True)
pl.Path('TENTH_POINT').mkdir(parents=False, exist_ok=True)
path = "TENTH_POINT/companies_decade_revenue.csv"
companies_df.to_csv(path_or_buf = path)
print("Succesfully written normalized table to ", path)

Succesfully written normalized table to  TENTH_POINT/companies_decade_revenue.csv


Now we can group by the *name* and the *decade*. Finally the `sum()` method provides us with the total revenue of the company in that decade.

In [25]:
companies_decade_revenue = companies_df.groupby(['name','decade'])['revenue'].sum()
companies_decade_revenue.head()

name                             decade
"DIA" Productions GmbH & Co. KG  200       11087731
1.85 Films                       201          12252
10 West Studios                  201              0
100 Bares                        200        3773982
                                 201        2666666
Name: revenue, dtype: int64

# Eleventh request
## For each decade, compute the company with maximum revenue
To solve this task we can proceed by identifying the companies having the maximum value of revenue grouped by *decade* in the normalized table created for the tenth request. The `idxmax()` outputs the indexes of such companies as a `pd.Series` object.

In [28]:
eleventh = pd.read_csv("TENTH_POINT/companies_decade_revenue.csv")
id_max_revenue = eleventh.groupby('decade', as_index = False)['revenue'].idxmax()

In [29]:
eleventh.iloc[id_max_revenue][['decade','name','revenue']]

,decade,name,revenue
13415,191,Triangle Film Corporation,4197375
13509,192,Metro-Goldwyn-Mayer (MGM),22000000
11961,193,Selznick International Pictures,200088229
13193,194,Walt Disney Productions,267447150
11365,195,Twentieth Century Fox Film Corporation,36000000
9648,196,Metro-Goldwyn-Mayer (MGM),111858363
9640,197,Lucasfilm,387699003
9788,198,Universal Pictures,396455277
891,199,Lucasfilm,924317558
1275,200,Pixar Animation Studios,940335536


# Twelth request
## In each year, how many movies have revenue smaller than the budget?
Again we begin by assigning a new column to the **movies** table to store the year of release for each movie.

In [9]:
twelth = movies.dropna(subset=['release_date']).assign(year = pd.DatetimeIndex(movies.dropna(subset=['release_date'])['release_date']).year)
twelth['year'] = twelth['year'].apply(lambda x: int(x))

We can obtain all movies that have revenue smaller than the budget with conditional indexing. Then we proceed grouping on the *year* attribute and finally obtaining the number of movies for each year through the `size()` method.

In [10]:
unsuccesful = twelth[twelth['revenue'] < twelth['budget']].groupby('year').size()
unsuccesful.head()

year
1927    1
1947    1
1948    1
1954    1
1962    1
dtype: int64